In [2]:
import pandas as pd
import numpy as np
import os
from transformers import BertModel, BertTokenizer
import torch
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
# from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

import spacy
nlp = spacy.load('en')

import textsimilarity
from textsimilarity.build_semantic_graph.build_tree import build_tree
from textsimilarity.build_semantic_graph.prune_and_merge_tree import prune
from textsimilarity.build_semantic_graph.rearrange_tree import rearrange
from textsimilarity.build_semantic_graph.build_graph import get_graph
from textsimilarity.build_semantic_graph.merge_graph import merge
from textsimilarity.build_semantic_graph.build_semantic_graph import merge_dp_coref

from textsimilarity.preprocess import get_coref_and_dp


stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jiajinghu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
TWITTER_RAW_DIR = '../data/twitter/raw'
TWITTER_INTERIM_DIR = '../data/twitter/interim'
TWITTER_PROCESSED_DIR = '../data/twitter/processed'

In [4]:
MAX_LENGTH=128
BATCH_SIZE=32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
stop_words = set(stopwords.words('english')) 

In [5]:
def pad_to_max_length(ids, MAX_LENGTH):
    ids = torch.tensor(pad_sequence(ids, batch_first=True))
    columns = ids.shape[1]
    output_ids = torch.zeros(ids.shape[0], MAX_LENGTH)
    output_ids[:, :columns] = ids
    return output_ids

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

In [7]:
vocab = tokenizer.get_vocab()
vocab_list = list(vocab.keys())

In [8]:
twitter_train = pd.read_csv(os.path.join(TWITTER_RAW_DIR, 'train.csv'))
twitter_test = pd.read_csv(os.path.join(TWITTER_RAW_DIR, 'test.csv'))

twitter_graph_train = np.load(os.path.join(TWITTER_PROCESSED_DIR, 'graphs_train.npy'), allow_pickle=True)
twitter_graph_test = np.load(os.path.join(TWITTER_PROCESSED_DIR, 'graphs_test.npy'), allow_pickle=True)

twitter_dp_train = np.load(os.path.join(TWITTER_INTERIM_DIR, 'dp_train.npy'), allow_pickle=True)
twitter_dp_test = np.load(os.path.join(TWITTER_INTERIM_DIR, 'dp_test.npy'), allow_pickle=True)

twitter_coref_train = np.load(os.path.join(TWITTER_INTERIM_DIR, 'coref_train.npy'), allow_pickle=True)
twitter_coref_test = np.load(os.path.join(TWITTER_INTERIM_DIR, 'coref_test.npy'), allow_pickle=True)

In [9]:
twitter_coref_train = list(map(lambda x: [j for n in x for j in n.split()], twitter_coref_train))
twitter_coref_test = list(map(lambda x: [j for n in x for j in n.split()], twitter_coref_test))

#### Get Bert Embeddings

In [10]:
input_token_ids_train = list(map(lambda x: torch.tensor(tokenizer.convert_tokens_to_ids([j if j in vocab_list else 
                                                                                         tokenizer.tokenize(j)[0] for j in x])), 
                                 twitter_coref_train))

input_token_ids_test = list(map(lambda x: torch.tensor(tokenizer.convert_tokens_to_ids([j if j in vocab_list else 
                                                                                         tokenizer.tokenize(j)[0] for j in x])), 
                                twitter_coref_test))

input_token_ids_train = pad_to_max_length(input_token_ids_train, MAX_LENGTH)
input_token_ids_test = pad_to_max_length(input_token_ids_test, MAX_LENGTH)


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [63]:
dataset_train = TensorDataset(input_token_ids_train)
dataloader_train = DataLoader(
            dataset_train,
            sampler = SequentialSampler(dataset_train),
            batch_size = BATCH_SIZE 
        )

bert_embeddings_train = []
with torch.no_grad():
    for b in dataloader_train:
        bert_embeddings_train.extend(bert_model(b[0].type(torch.LongTensor).to(device))[0].cpu().numpy())

bert_embeddings_train = np.array(bert_embeddings_train)
np.save(os.path.join(TWITTER_INTERIM_DIR, 'bert_embeddings_train.npy'), bert_embeddings_train)



dataset_test = TensorDataset(input_token_ids_test)
dataloader_test = DataLoader(
            dataset_test,
            sampler = SequentialSampler(dataset_test),
            batch_size = BATCH_SIZE 
        )

bert_embeddings_test = []
with torch.no_grad():
    for b in dataloader_test:
        bert_embeddings_test.extend(bert_model(b[0].type(torch.LongTensor).to(device))[0].cpu().numpy())

bert_embeddings_test = np.array(bert_embeddings_test)
np.save(os.path.join(TWITTER_INTERIM_DIR, 'bert_embeddings_test.npy'), bert_embeddings_test)

In [10]:
bert_embeddings_train = np.load(os.path.join(TWITTER_INTERIM_DIR, 'bert_embeddings_train.npy'))
bert_embeddings_test = np.load(os.path.join(TWITTER_INTERIM_DIR, 'bert_embeddings_test.npy'))

#### Get Word2vec embeddings

In [11]:
def get_word2vec_embeddings(doc):
    nlp_doc = nlp(doc)
    emb = np.array([nlp_doc[i].vector for i in range(len(doc.split()))])
    return emb

training_sents = list(map(lambda x: ' '.join(x), twitter_coref_train))
word2vec_embeddings_train = np.array(list(map(get_word2vec_embeddings, training_sents)))
np.save(os.path.join(TWITTER_INTERIM_DIR, 'word2vec_embeddings_train.npy'), word2vec_embeddings_train)

testing_sents = list(map(lambda x: ' '.join(x), twitter_coref_test))
word2vec_embeddings_test = np.array(list(map(get_word2vec_embeddings, testing_sents)))
np.save(os.path.join(TWITTER_INTERIM_DIR, 'word2vec_embeddings_test.npy'), word2vec_embeddings_test)

In [12]:
word2vec_embeddings_train = np.load(os.path.join(TWITTER_INTERIM_DIR, 'word2vec_embeddings_train.npy'), 
                                    allow_pickle=True)
word2vec_embeddings_test = np.load(os.path.join(TWITTER_INTERIM_DIR, 'word2vec_embeddings_test.npy'),
                                  allow_pickle=True)

In [11]:
doc1 = 'obama speaks to the media in illinois'.split(' ')
emb1 = bert_model(torch.tensor([tokenizer.convert_tokens_to_ids(doc1)]))[0].squeeze().detach().numpy()

doc2 = 'the president greet the press in chicago'.split(' ')
emb2 = bert_model(torch.tensor([tokenizer.convert_tokens_to_ids(doc2)]))[0].squeeze().detach().numpy()

doc3 = 'the band gave a concert in japan'.split(' ')
emb3 = bert_model(torch.tensor([tokenizer.convert_tokens_to_ids(doc3)]))[0].squeeze().detach().numpy()

In [13]:
def cosine_similarity(vec1, vec2):
    dot = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot/(norm1*norm2)

In [14]:
def get_wms(doc1, doc2, emb1, emb2):
    # we treat doc1 as the query document
    result1 = [(word, emb) for word, emb in zip(doc1,emb1) if (word not in stop_words) and (not np.all(emb==0))]
    document1 = [x[0] for x in result1]
    embedding1 = [x[1] for x in result1]
    
    result2 = [(word, emb) for word, emb in zip(doc2,emb2) if (word not in stop_words) and (not np.all(emb==0))]
    document2 = [x[0] for x in result2]
    embedding2 = [x[1] for x in result2]
    
    s1 = set(document1)
    s2 = set(document2)
    
    d2 = {}
    for s in s2:
        d2[s] = document2.count(s)/len(document2)
    
    sent_sim = []
    for i2, e2 in enumerate(embedding2):
        word_sim = []
        for e1 in embedding1:
            word_sim.append(cosine_similarity(e1, e2))
#         print(document2[i2], word_sim)
        sent_sim.append(max(word_sim)*d2[document2[i2]])
    return sum(sent_sim)

#### WMD using bert embeddings

In [88]:
%%time
sim = np.zeros((bert_embeddings_train.shape[0], bert_embeddings_train.shape[0]))
for i in range(sim.shape[0]):
    for j in range(i+1, sim.shape[0]):
        sim[i,j] = get_wms(twitter_coref_train[i], twitter_coref_train[j], 
                           bert_embeddings_train[i], bert_embeddings_train[j], vocab_list)
    print(i)
    
sim = sim + sim.T + np.identity(sim.shape[0])
np.save(os.path.join(TWITTER_PROCESSED_DIR, 'wms_train.npy'), sim)
np.save(os.path.join(TWITTER_PROCESSED_DIR, 'wmd_train.npy'), 1-sim)

In [56]:
%%time
sim_test = np.zeros((bert_embeddings_test.shape[0], bert_embeddings_train.shape[0]))
for i in range(sim_test.shape[0]):
    for j in range(sim.shape[0]):
        sim_test[i,j] = get_wms(twitter_coref_train[j], twitter_coref_test[i], 
                           bert_embeddings_train[j], bert_embeddings_test[i], vocab_list)
    print(i)
    
np.save(os.path.join(TWITTER_PROCESSED_DIR, 'wms_test.npy'), sim_test)
np.save(os.path.join(TWITTER_PROCESSED_DIR, 'wmd_test.npy'), 1-sim_test)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [86]:
knn = KNeighborsClassifier(metric="precomputed")
knn.fit(1-sim, twitter_train.label)

pred_test = knn.predict(1-sim_test)
accuracy_score(twitter_test.label, pred_test)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='precomputed',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [89]:
pred_dev = knn.predict(1-sim_dev)
accuracy_score(twitter_dev.label, pred_dev)

0.6903553299492385

In [90]:
pred_test = knn.predict(1-sim_test)
accuracy_score(twitter_test.label, pred_test)

0.6834170854271356

#### WMD using word2vec embeddings

In [91]:
%%time
sim = np.zeros((word2vec_embeddings_train.shape[0], word2vec_embeddings_train.shape[0]))
for i in range(sim.shape[0]):
    for j in range(i+1, sim.shape[0]):
        sim[i,j] = get_wms(twitter_coref_train[i], twitter_coref_train[j], 
                           word2vec_embeddings_train[i], word2vec_embeddings_train[j])
    print(i)
    
sim = sim + sim.T + np.identity(sim.shape[0])
np.save(os.path.join(TWITTER_PROCESSED_DIR, 'wms_word2vec_train.npy'), sim)
np.save(os.path.join(TWITTER_PROCESSED_DIR, 'wmd_word2vec_train.npy'), 1-sim)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [92]:
%%time
sim_test = np.zeros((word2vec_embeddings_test.shape[0], word2vec_embeddings_train.shape[0]))
for i in range(sim_test.shape[0]):
    for j in range(sim.shape[0]):
        sim_test[i,j] = get_wms(twitter_coref_train[j], twitter_coref_test[i], 
                           word2vec_embeddings_train[j], word2vec_embeddings_test[i])
    print(i)
    
np.save(os.path.join(TWITTER_PROCESSED_DIR, 'wms_word2vec_test.npy'), sim_test)
np.save(os.path.join(TWITTER_PROCESSED_DIR, 'wmd_word2vec_test.npy'), 1-sim_test)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [44]:
dist_train = np.load(os.path.join(TWITTER_PROCESSED_DIR, 'wmd_word2vec_train.npy'))
dist_test = np.load(os.path.join(TWITTER_PROCESSED_DIR, 'wmd_word2vec_test.npy'))

knn = KNeighborsClassifier(metric="precomputed")
knn.fit(dist_train, twitter_train.label)

pred_test = knn.predict(dist_test)
accuracy_score(twitter_test.label, pred_test)

0.7171717171717171

In [37]:
wmd_incorrect_idx = twitter_test[twitter_test.label != pred_test].index.values
wmd_incorrect = twitter_test.take(wmd_incorrect_idx)
wmd_incorrect

,label,sentence
1,"""negative""",after i don't know how many years
8,"""positive""",validated that microsoft iscsi target in win s...
12,"""neutral""",rt ecommnewsupdate steve yegge's google plus r...
21,"""negative""",asked siri where's baby lisa and was told sorry
24,"""positive""",i'm a current blackberry user
34,"""neutral""",gonna crash out pretty tired
38,"""neutral""",saw that apple is covered with msgs 4 steve jo...
39,"""neutral""",siri is cool but more importantly handy and fu...
41,"""negative""",microsoft windows media centre gui fail majorfail
45,"""positive""",rt ashercfo yahoo up 4 sale again yrs back mic...


In [48]:
wmd_incorrect_pred = pd.DataFrame(pred_test).take(wmd_incorrect_idx)
wmd_incorrect_pred

,0
1,"""neutral"""
8,"""neutral"""
12,"""negative"""
21,"""neutral"""
24,"""negative"""
34,"""negative"""
38,"""negative"""
39,"""negative"""
41,"""neutral"""
45,"""neutral"""


## Get similarit with graph

In [50]:
def get_node_embeddings(graph, embeddings):
    node_features = []
    for node in graph['nodes']:
        node_features.append(embeddings[[node['index']]].mean(axis=0))            
    return np.array(node_features)

In [54]:
def get_graph_similarity(graph1, graph2, emb1, emb2, threshold):
    # we treat graph1 as the query document
    embedding1 = get_node_embeddings(graph1, emb1)
    embedding2 = get_node_embeddings(graph2, emb2)
    
    w_match = 1
    w_nonmatch = 0.5
    
    graph_sim = []
    node_match = []
    for i2, e2 in enumerate(embedding2):
        node_sim = []
        for e1 in embedding1:
            if (np.all(e1==0))|(np.all(e2==0)):
                node_sim.append(0)
            else:
                node_sim.append(cosine_similarity(e1, e2))
        max_sim_node = np.argmax(node_sim)
        if max(node_sim) >= threshold:
            node_match.append((max_sim_node, i2))
        graph_sim.append(max(node_sim))
        
#     print('node match: ', node_match)
    if len(node_match) <= 1:
        return sum(graph_sim)*w_nonmatch
    else:
        weights = np.zeros(len(graph_sim)) + w_nonmatch
        for i in range(len(node_match)):
            for j in range(i, len(node_match)):
                n1_g1 = node_match[i][0]
                n1_g2 = node_match[i][1]
                n2_g1 = node_match[j][0]
                n2_g2 = node_match[j][1]
                
                n1_n2_g1 = graph1['edges'][n1_g1][n2_g1]
                n2_n1_g1 = graph1['edges'][n2_g1][n1_g1]

                n1_n2_g2 = graph2['edges'][n1_g2][n2_g2]
                n2_n1_g2 = graph2['edges'][n2_g2][n1_g2]

                if ((n1_n2_g1!='')&(n1_n2_g1 == n1_n2_g2))|((n2_n1_g1!='')&(n2_n1_g1 == n2_n1_g2)):
                    weights[n1_g2] += (w_match-w_nonmatch)
                    weights[n2_g2] += (w_match-w_nonmatch)
#                     print(n1_g2)
#                     print(n2_g2)
                    
        return np.dot(graph_sim, weights)/len(graph2), weights

In [49]:
# # bert embeddings
# %%time
# threshold = 0
# g_sim_train = np.zeros((bert_embeddings_train.shape[0], bert_embeddings_train.shape[0]))
# num_edge_match = 0
# for i in range(g_sim_train.shape[0]):
#     for j in range(i+1, g_sim_train.shape[0]):
#         sim = get_graph_similarity(twitter_graph_train[i], twitter_graph_train[j], 
#                            bert_embeddings_train[i], bert_embeddings_train[j], threshold)
#         if type(sim) == tuple:
#             g_sim_train[i,j] = sim[0]
#             if max(sim[1]) > 0.5:
#                 num_edge_match += 1
#         else:
#             g_sim_train[i,j] = sim
#     print(i)
    
# g_sim_train = g_sim_train + g_sim_train.T
# np.save(os.path.join(TWITTER_PROCESSED_DIR, f'g_sim_train_bert_threshold{threshold}.npy'), g_sim_train)
# np.save(os.path.join(TWITTER_PROCESSED_DIR, f'g_dist_train_bert_threshold{threshold}.npy'), 1-g_sim_train)
# print(num_edge_match)

161836


#### bert embeddings similarity

In [67]:
%%time
edge_matches = []
for threshold in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    g_sim_train = np.zeros((bert_embeddings_train.shape[0], bert_embeddings_train.shape[0]))
    num_edge_match = 0
    for i in range(g_sim_train.shape[0]):
        for j in range(i+1, g_sim_train.shape[0]):
            sim = get_graph_similarity(twitter_graph_train[i], twitter_graph_train[j], 
                               bert_embeddings_train[i], bert_embeddings_train[j], threshold)
            if type(sim) == tuple:
                g_sim_train[i,j] = sim[0]
                if max(sim[1]) > 0.5:
                    num_edge_match += 1
            else:
                g_sim_train[i,j] = sim
        print(i)
    
    edge_matches.append(num_edge_match)
    g_sim_train = g_sim_train + g_sim_train.T + np.identity(g_sim_train.shape[0])
    np.save(os.path.join(TWITTER_PROCESSED_DIR, f'g_sim_train_bert_threshold{threshold}.npy'), g_sim_train)
    np.save(os.path.join(TWITTER_PROCESSED_DIR, f'g_dist_train_bert_threshold{threshold}.npy'), 1-g_sim_train)
    print(num_edge_match)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  after removing the cwd from sys.path.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

161836
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275

161836
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275

161815
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275

161350
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275

156387
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275

126986
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275

46036
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275


528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777


In [68]:
%%time
edge_matches_test = []
for threshold in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    g_sim_test = np.zeros((bert_embeddings_test.shape[0], bert_embeddings_train.shape[0]))
    num_edge_match_test = 0

    for i in range(g_sim_test.shape[0]):
        for j in range(g_sim_train.shape[0]):
            sim = get_graph_similarity(twitter_graph_train[j], twitter_graph_test[i], 
                               bert_embeddings_train[j], bert_embeddings_test[i], threshold)
            if type(sim) == tuple:
                g_sim_test[i,j] = sim[0]
                if max(sim[1]) > 0.5:
                    num_edge_match_test += 1
            else:
                g_sim_test[i,j] = sim
        print(i)
    edge_matches_test.append(num_edge_match_test)
    np.save(os.path.join(TWITTER_PROCESSED_DIR, f'g_sim_test_bert_threshold{threshold}.npy'), g_sim_test)
    np.save(os.path.join(TWITTER_PROCESSED_DIR, f'g_dist_test_bert_threshold{threshold}.npy'), 1-g_sim_test)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  after removing the cwd from sys.path.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373


644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
12

399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648


In [12]:
accuracy = []
for threshold in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    dist_train = np.load(os.path.join(TWITTER_PROCESSED_DIR, f'g_dist_train_bert_threshold{threshold}.npy'))
    dist_test = np.load(os.path.join(TWITTER_PROCESSED_DIR, f'g_dist_test_bert_threshold{threshold}.npy'))
    knn = KNeighborsClassifier(metric="precomputed")
    knn.fit(dist_train, twitter_train.label)
    pred_test = knn.predict(dist_test)
    acc = accuracy_score(twitter_test.label, pred_test)
    accuracy.append(acc)
    print(acc)

0.6792929292929293
0.6792929292929293
0.6792929292929293
0.6792929292929293
0.6767676767676768
0.6742424242424242
0.678030303030303
0.6616161616161617
0.6616161616161617
0.6805555555555556


#### word2vec embeddings similarity

In [51]:
get_graph_similarity(twitter_graph_train[0], twitter_graph_train[21], 
                               word2vec_embeddings_train[0], word2vec_embeddings_train[21], threshold)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  after removing the cwd from sys.path.


(1.1863612830638885, array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]))

In [62]:
edge_matches = []
threshold=0
g_sim_train = np.zeros((word2vec_embeddings_train.shape[0], word2vec_embeddings_train.shape[0]))
num_edge_match = 0
for i in range(g_sim_train.shape[0]):
    for j in range(i+1, g_sim_train.shape[0]):
        sim = get_graph_similarity(twitter_graph_train[i], twitter_graph_train[j], 
                           word2vec_embeddings_train[i], word2vec_embeddings_train[j], threshold)
#             if np.count_nonzero(np.isnan(sim))
        if type(sim) == tuple:
            g_sim_train[i,j] = sim[0]
            if max(sim[1]) > 0.5:
                num_edge_match += 1
        else:
            g_sim_train[i,j] = sim
        if np.isnan(g_sim_train[i,j]):
                print('j: ', j)
    print(i)

edge_matches.append(num_edge_match)
g_sim_train = g_sim_train + g_sim_train.T + np.identity(g_sim_train.shape[0])

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  after removing the cwd from sys.path.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [69]:
np.sum(np.isnan(1-g_sim_train))

0

In [65]:
np.save(os.path.join(TWITTER_PROCESSED_DIR, f'g_sim_train_word2vec_threshold{threshold}.npy'), g_sim_train)
np.save(os.path.join(TWITTER_PROCESSED_DIR, f'g_dist_train_word2vec_threshold{threshold}.npy'), 1-g_sim_train)

In [60]:
%%time
for threshold in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    edge_matches = []
    g_sim_train = np.zeros((word2vec_embeddings_train.shape[0], word2vec_embeddings_train.shape[0]))
    num_edge_match = 0
    for i in range(g_sim_train.shape[0]):
        for j in range(i+1, g_sim_train.shape[0]):
            sim = get_graph_similarity(twitter_graph_train[i], twitter_graph_train[j], 
                               word2vec_embeddings_train[i], word2vec_embeddings_train[j], threshold)
#             if np.count_nonzero(np.isnan(sim))
            if type(sim) == tuple:
                g_sim_train[i,j] = sim[0]
                if max(sim[1]) > 0.5:
                    num_edge_match += 1
            else:
                g_sim_train[i,j] = sim
            if np.isnan(g_sim_train[i,j]):
                print('j: ', j)
        print(i)
    
    edge_matches.append(num_edge_match)
    g_sim_train = g_sim_train + g_sim_train.T + np.identity(g_sim_train.shape[0])
    np.save(os.path.join(TWITTER_PROCESSED_DIR, f'g_sim_train_word2vec_threshold{threshold}.npy'), g_sim_train)
    np.save(os.path.join(TWITTER_PROCESSED_DIR, f'g_dist_train_word2vec_threshold{threshold}.npy'), 1-g_sim_train)
    print(num_edge_match)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  after removing the cwd from sys.path.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/magics/execution.py", line 1315, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 10, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/ultratb.py", line 316, in wra

TypeError: object of type 'NoneType' has no len()

In [75]:
edge_matches_test = []
g_sim_test = np.zeros((word2vec_embeddings_test.shape[0], word2vec_embeddings_train.shape[0]))
num_edge_match_test = 0

for i in range(g_sim_test.shape[0]):
    for j in range(g_sim_train.shape[0]):
        sim = get_graph_similarity(twitter_graph_train[j], twitter_graph_test[i], 
                           word2vec_embeddings_train[j], word2vec_embeddings_test[i], threshold)
        if type(sim) == tuple:
            g_sim_test[i,j] = sim[0]
            if max(sim[1]) > 0.5:
                num_edge_match_test += 1
        else:
            g_sim_test[i,j] = sim
    print(i)
edge_matches_test.append(num_edge_match_test)
np.save(os.path.join(TWITTER_PROCESSED_DIR, f'g_sim_test_word2vec_threshold{threshold}.npy'), g_sim_test)
np.save(os.path.join(TWITTER_PROCESSED_DIR, f'g_dist_test_word2vec_threshold{threshold}.npy'), 1-g_sim_test)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  after removing the cwd from sys.path.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [76]:
np.isnan(g_sim_test).sum()

0

In [ ]:
%%time

for threshold in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    edge_matches_test = []
    g_sim_test = np.zeros((word2vec_embeddings_test.shape[0], word2vec_embeddings_train.shape[0]))
    num_edge_match_test = 0

    for i in range(g_sim_test.shape[0]):
        for j in range(g_sim_train.shape[0]):
            sim = get_graph_similarity(twitter_graph_train[j], twitter_graph_test[i], 
                               word2vec_embeddings_train[j], word2vec_embeddings_test[i], threshold)
            if type(sim) == tuple:
                g_sim_test[i,j] = sim[0]
                if max(sim[1]) > 0.5:
                    num_edge_match_test += 1
            else:
                g_sim_test[i,j] = sim
        print(i)
    edge_matches_test.append(num_edge_match_test)
    np.save(os.path.join(TWITTER_PROCESSED_DIR, f'g_sim_test_word2vec_threshold{threshold}.npy'), g_sim_test)
    np.save(os.path.join(TWITTER_PROCESSED_DIR, f'g_dist_test_word2vec_threshold{threshold}.npy'), 1-g_sim_test)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  after removing the cwd from sys.path.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
0
1
2
3
4
5
6
7
8
9


302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551


In [50]:
accuracy = []
for threshold in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    dist_train = np.load(os.path.join(TWITTER_PROCESSED_DIR, f'g_dist_train_word2vec_threshold{threshold}.npy'))
    dist_test = np.load(os.path.join(TWITTER_PROCESSED_DIR, f'g_dist_test_word2vec_threshold{threshold}.npy'))
    knn = KNeighborsClassifier(metric="precomputed")
    knn.fit(dist_train, twitter_train.label)
    pred_test = knn.predict(dist_test)
    acc = accuracy_score(twitter_test.label, pred_test)
    accuracy.append(acc)
    print(acc)

0.672979797979798
0.6691919191919192
0.6704545454545454
0.672979797979798
0.6742424242424242
0.6818181818181818
0.6805555555555556
0.6742424242424242
0.6590909090909091
0.678030303030303


In [39]:
graph_incorrect_idx = twitter_test[twitter_test.label != pred_test].index.values
graph_incorrect = twitter_test.take(graph_incorrect_idx)
graph_incorrect

,label,sentence
1,"""negative""",after i don't know how many years
2,"""neutral""",towhat technological extensivity microsoft wil...
6,"""neutral""",microsoft releases developer preview of projec...
8,"""positive""",validated that microsoft iscsi target in win s...
11,"""neutral""",iphone 4s first weekend sales top four million...
21,"""negative""",asked siri where's baby lisa and was told sorry
24,"""positive""",i'm a current blackberry user
25,"""neutral""",ios5 needed to include a landscape mode for re...
35,"""neutral""",coatgal apple considering his circumstances su...
41,"""negative""",microsoft windows media centre gui fail majorfail


In [77]:
only_wmd_incorrect = twitter_test.take(list(set(wmd_incorrect_idx)-set(graph_incorrect_idx))).sort_index()
only_wmd_incorrect

,label,sentence
12,"""neutral""",rt ecommnewsupdate steve yegge's google plus r...
34,"""neutral""",gonna crash out pretty tired
38,"""neutral""",saw that apple is covered with msgs 4 steve jo...
39,"""neutral""",siri is cool but more importantly handy and fu...
61,"""negative""",google cloud service blackouts threaten cloud ...
62,"""negative""",steelo254 yea i pre order through apple and th...
92,"""negative""",rt anup pillai how apple eclipsed microsoft
95,"""negative""",it would be great if apple would send my new p...
102,"""neutral""",will apple's mac ever properly be used in the ...
109,"""neutral""",i need 2 unfollow some of these wannabe g's rt...


In [80]:
only_graph_wrong = twitter_test.take(list(set(graph_incorrect_idx)-set(wmd_incorrect_idx))).sort_index()
only_graph_wrong

,label,sentence
2,"""neutral""",towhat technological extensivity microsoft wil...
6,"""neutral""",microsoft releases developer preview of projec...
11,"""neutral""",iphone 4s first weekend sales top four million...
25,"""neutral""",ios5 needed to include a landscape mode for re...
35,"""neutral""",coatgal apple considering his circumstances su...
48,"""neutral""",rt petapixel iphone 4s vs canon 5d mark ii a s...
51,"""neutral""",sigsource microsoft talent source program boas...
57,"""neutral""",wow jus made a twitter i reallly dnt kno how o...
70,"""positive""",apple loving the new ios5 upgrade for the iphone
71,"""neutral""",pinball fx2 offers free 'paranormal' table sta...


In [81]:
only_graph_wrong.label.value_counts()

"neutral"     78
"negative"    28
"positive"    22
Name: label, dtype: int64

In [85]:
wmd_incorrect_pred[0].value_counts()

"neutral"     105
"negative"     92
"positive"     27
Name: 0, dtype: int64

In [75]:
wmd_incorrect_pred.loc[102]

0    "negative"
Name: 102, dtype: object

In [97]:
twitter_test.loc[128].sentence

'so apple the new imessage is great and all'

In [52]:
set(wmd_incorrect_idx)-set(graph_incorrect_idx)

{12,
 34,
 38,
 39,
 61,
 62,
 92,
 95,
 102,
 109,
 114,
 121,
 128,
 134,
 135,
 142,
 146,
 148,
 150,
 164,
 171,
 200,
 214,
 223,
 233,
 240,
 241,
 248,
 259,
 280,
 294,
 299,
 307,
 333,
 341,
 354,
 360,
 369,
 370,
 378,
 384,
 406,
 412,
 425,
 430,
 434,
 435,
 440,
 465,
 475,
 479,
 484,
 486,
 502,
 503,
 512,
 519,
 520,
 525,
 534,
 538,
 542,
 543,
 558,
 561,
 564,
 566,
 570,
 572,
 574,
 575,
 596,
 604,
 607,
 620,
 628,
 635,
 638,
 665,
 667,
 672,
 681,
 685,
 689,
 691,
 717,
 725,
 736,
 749,
 754,
 761,
 763,
 775,
 777,
 778,
 783,
 786}

In [64]:
set(graph_incorrect_idx) - set(wmd_incorrect_idx)

{2,
 6,
 11,
 25,
 35,
 48,
 51,
 57,
 70,
 71,
 76,
 99,
 104,
 111,
 115,
 117,
 122,
 125,
 129,
 136,
 138,
 141,
 147,
 151,
 153,
 156,
 160,
 162,
 168,
 173,
 177,
 186,
 187,
 194,
 198,
 211,
 215,
 216,
 217,
 231,
 244,
 250,
 263,
 269,
 271,
 285,
 291,
 309,
 313,
 329,
 334,
 337,
 339,
 343,
 346,
 347,
 352,
 361,
 375,
 386,
 387,
 397,
 400,
 404,
 405,
 408,
 410,
 413,
 414,
 415,
 418,
 421,
 443,
 453,
 460,
 470,
 474,
 476,
 478,
 482,
 483,
 487,
 488,
 497,
 501,
 507,
 511,
 523,
 530,
 531,
 532,
 539,
 541,
 544,
 551,
 565,
 567,
 576,
 577,
 580,
 581,
 588,
 591,
 594,
 599,
 605,
 611,
 617,
 634,
 642,
 643,
 644,
 651,
 660,
 662,
 674,
 687,
 693,
 694,
 698,
 704,
 719,
 728,
 738,
 739,
 751,
 773,
 779}

In [104]:
twitter_graph_test[48]

{'nodes': [{'type': 'A',
   'dep': 'root',
   'pos': 'NOUN',
   'word': 'rt petapixel iphone',
   'index': [0, 1, 2]},
  {'type': 'A', 'dep': 'dep', 'pos': 'NOUN', 'word': '4s', 'index': [3]},
  {'type': 'A',
   'dep': 'dep',
   'pos': 'ADP',
   'word': 'vs canon',
   'index': [4, 5]},
  {'type': 'A', 'dep': 'dep', 'pos': 'NUM', 'word': '5d', 'index': [6]},
  {'type': 'A',
   'dep': 'dep',
   'pos': 'VERB',
   'word': 'mark ii',
   'index': [7, 8]},
  {'type': 'A', 'dep': 'prep', 'pos': 'ADP', 'word': 'by', 'index': [11]},
  {'type': 'A',
   'dep': 'pobj',
   'pos': 'NOUN',
   'word': 'a side',
   'index': [9, 10]},
  {'type': 'A', 'dep': 'pobj', 'pos': 'NOUN', 'word': 'side', 'index': [12]},
  {'type': 'A',
   'dep': 'pobj',
   'pos': 'NOUN',
   'word': 'comparison',
   'index': [13]},
  {'type': 'M',
   'dep': 'pobj',
   'pos': 'NOUN',
   'word': 'of 1080p hd video lt lt apple wow',
   'index': [14, 15, 16, 17, 18, 19, 20, 21]}],
 'edges': [['', '', '', '', '', '', '', '', '', ''],
 

In [35]:
doc1 = 'obama speaks to the media in illinois'
# emb1 = bert_model(torch.tensor([tokenizer.convert_tokens_to_ids(doc1.split(' '))]))[0].squeeze().detach().numpy()
nlp1 = nlp(doc1)
emb1 = np.array([nlp1[i].vector for i in range(len(doc1.split()))])
dp1 = get_coref_and_dp.get_dependency(doc1)
coref1 = get_coref_and_dp.get_coreference(doc1)
sents_data1 = {'dependency_parse':dp1, 'coreference':coref1}
sents_data1 = build_tree(sents_data1)
sents_data1 = {'sequence':sents_data1['words'], 'tree':prune(sents_data1['tree'], sents_data1['words'])}
sents_data1 = {'sequence': sents_data1['sequence'], 'tree': rearrange(sents_data1['tree'], sents_data1['sequence'])}
g1 = get_graph(sents_data1['tree'])
# emb1 = get_node_embeddings(g1, emb1)

doc2 = 'the president greet the press in chicago'
# emb2 = bert_model(torch.tensor([tokenizer.convert_tokens_to_ids(doc2.split(' '))]))[0].squeeze().detach().numpy()
nlp2 = nlp(doc2)
emb2 = np.array([nlp2[i].vector for i in range(len(doc2.split()))])
dp2 = get_coref_and_dp.get_dependency(doc2)
coref2 = get_coref_and_dp.get_coreference(doc2)
sents_data2 = {'dependency_parse':dp2, 'coreference':coref2}
sents_data2 = build_tree(sents_data2)
sents_data2 = {'sequence':sents_data2['words'], 'tree':prune(sents_data2['tree'], sents_data2['words'])}
sents_data2 = {'sequence': sents_data2['sequence'], 'tree': rearrange(sents_data2['tree'], sents_data2['sequence'])}
g2 = get_graph(sents_data2['tree'])
# emb2 = get_node_embeddings(g2, emb2)



In [36]:
get_graph_similarity(g1, g2, emb1, emb2, 0)

node match:  [(0, 0), (2, 1), (2, 2), (3, 3)]


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  after removing the cwd from sys.path.


(1.2739520817995071, array([1.5, 0.5, 0.5, 1.5]))

### Weighted average between WMD and graph similarity

#### Word2vec

In [27]:
wmd_word2vec_train = np.load(os.path.join(TWITTER_PROCESSED_DIR, 'wmd_word2vec_train.npy'))
wmd_wrd2vec_test = np.load(os.path.join(TWITTER_PROCESSED_DIR, 'wmd_word2vec_test.npy'))

In [28]:
accuracy = []
for threshold in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    print('node match threshold is: ', threshold)
    for weight in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
#         print('graph distance weight is: ', weight)
        dist_graph_train = np.load(os.path.join(TWITTER_PROCESSED_DIR, f'g_dist_train_word2vec_threshold{threshold}.npy'))
        dist_graph_test = np.load(os.path.join(TWITTER_PROCESSED_DIR, f'g_dist_test_word2vec_threshold{threshold}.npy'))
        
        dist_train_avg = (1-weight)*wmd_word2vec_train + weight*dist_graph_train
        dist_test_avg = (1-weight)*wmd_wrd2vec_test + weight*dist_graph_test
        
        knn = KNeighborsClassifier(metric="precomputed")
        knn.fit(dist_train_avg, twitter_train.label)
        pred_test = knn.predict(dist_test_avg)
        acc = accuracy_score(twitter_test.label, pred_test)
        accuracy.append(acc)
        print(acc)
    print('--------------------------------------------------')

node match threshold is:  0
0.7171717171717171
0.6856060606060606
0.6957070707070707
0.6856060606060606
0.6792929292929293
0.6666666666666666
0.6704545454545454
0.6654040404040404
0.6691919191919192
0.6704545454545454
0.672979797979798
--------------------------------------------------
node match threshold is:  0.1
0.7171717171717171
0.6881313131313131
0.6957070707070707
0.6843434343434344
0.678030303030303
0.6628787878787878
0.6666666666666666
0.6641414141414141
0.6666666666666666
0.6704545454545454
0.6691919191919192
--------------------------------------------------
node match threshold is:  0.2
0.7171717171717171
0.6893939393939394
0.6944444444444444
0.6881313131313131
0.6792929292929293
0.6628787878787878
0.6654040404040404
0.6641414141414141
0.6666666666666666
0.6704545454545454
0.6704545454545454
--------------------------------------------------
node match threshold is:  0.3
0.7171717171717171
0.6919191919191919
0.696969696969697
0.6906565656565656
0.6843434343434344
0.66414141

#### BERT

In [38]:
wmd_bert_train = np.load(os.path.join(TWITTER_PROCESSED_DIR, 'wmd_bert_train.npy'))
wmd_bert_test = np.load(os.path.join(TWITTER_PROCESSED_DIR, 'wmd_bert_test.npy'))

In [39]:
accuracy = []
for threshold in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    print('node match threshold is: ', threshold)
    for weight in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
#         print('graph distance weight is: ', weight)
        dist_graph_train = np.load(os.path.join(TWITTER_PROCESSED_DIR, f'g_dist_train_bert_threshold{threshold}.npy'))
        dist_graph_test = np.load(os.path.join(TWITTER_PROCESSED_DIR, f'g_dist_test_bert_threshold{threshold}.npy'))
        
        dist_train_avg = (1-weight)*wmd_bert_train + weight*dist_graph_train
        dist_test_avg = (1-weight)*wmd_bert_test + weight*dist_graph_test
        
        knn = KNeighborsClassifier(metric="precomputed")
        knn.fit(dist_train_avg, twitter_train.label)
        pred_test = knn.predict(dist_test_avg)
        acc = accuracy_score(twitter_test.label, pred_test)
        accuracy.append(acc)
        print(acc)
    print('--------------------------------------------------')

node match threshold is:  0
0.6161616161616161
0.6553030303030303
0.6679292929292929
0.6679292929292929
0.6666666666666666
0.6616161616161617
0.6616161616161617
0.6666666666666666
0.672979797979798
0.6742424242424242
0.6792929292929293
--------------------------------------------------
node match threshold is:  0.1
0.6161616161616161
0.6553030303030303
0.6679292929292929
0.6679292929292929
0.6666666666666666
0.6616161616161617
0.6616161616161617
0.6666666666666666
0.672979797979798
0.6742424242424242
0.6792929292929293
--------------------------------------------------
node match threshold is:  0.2
0.6161616161616161
0.6553030303030303
0.6679292929292929
0.6679292929292929
0.6666666666666666
0.6616161616161617
0.6616161616161617
0.6666666666666666
0.672979797979798
0.6742424242424242
0.6792929292929293
--------------------------------------------------
node match threshold is:  0.3
0.6161616161616161
0.6553030303030303
0.6679292929292929
0.6679292929292929
0.6666666666666666
0.66161616